# Lecture 13-15

- extracts from 13-15
- good to have extra tools in your toolkit
- connecting duckdb to python
- connecting duckdb to pyhton


In [2]:
# no need for pathlib in jupiter , then 'r' for read

# Open and save the sql code
with open("sql/ingestion.sql", 'r') as sql_script:
    sql_code = sql_script.read()

print(sql_code)

CREATE SCHEMA IF NOT EXISTS staging;


CREATE TABLE IF NOT EXISTS staging.students AS (
    SELECT
        *
    FROM
        read_csv_auto('data/students.csv')
);


CREATE TABLE IF NOT EXISTS staging.teachers AS (
    SELECT
        *
    FROM
        read_csv_auto('data/teachers.csv')
);


CREATE TABLE IF NOT EXISTS staging.housing_sales AS (
    SELECT
        *
    FROM
        read_csv_auto('data/housing_sales.csv')
);


CREATE TABLE IF NOT EXISTS staging.neighborhoods AS (
    SELECT
        *
    FROM
        read_csv_auto('data/neighborhoods.csv')
);


CREATE TABLE IF NOT EXISTS staging.categories AS (
    SELECT
        *
    FROM
        read_csv_auto('data/categories.csv')
);


CREATE TABLE IF NOT EXISTS staging.sales AS (
    SELECT
        *
    FROM
        read_csv_auto('data/sales.csv')
);


## With the statement to connect to a duckdb

- eith handles connections and closes it down properly
- if this database doesnt exist it will create it for us


In [3]:
# connect to duckdb database and execute the sql - code
import duckdb

with duckdb.connect("data/lecture_13_15.duckdb") as conn:
    conn.execute(sql_code)
    describe = conn.execute("DESC;").df()

describe

,database,schema,name,column_names,column_types,temporary
0,lecture_13_15,staging,categories,"[category_id, category_name, parent_category_id]","[BIGINT, VARCHAR, BIGINT]",False
1,lecture_13_15,staging,housing_sales,"[sale_id, neighborhood_id, address, date_of_sa...","[VARCHAR, BIGINT, VARCHAR, DATE, VARCHAR, BIGI...",False
2,lecture_13_15,staging,neighborhoods,"[neighborhood_id, name, city, postal_code, med...","[BIGINT, VARCHAR, VARCHAR, BIGINT, BIGINT, BIG...",False
3,lecture_13_15,staging,sales,"[product_id, product_name, category_id, quanti...","[VARCHAR, VARCHAR, BIGINT, BIGINT, BIGINT]",False
4,lecture_13_15,staging,students,"[student_id, grade_math, class_name]","[VARCHAR, BIGINT, VARCHAR]",False
5,lecture_13_15,staging,teachers,"[class_name, teacher_name]","[VARCHAR, VARCHAR]",False


In [4]:
# doesnt work as the  connection is closed
# duckdb.sql("FROM staging.students;")

In [5]:
from utils import query_database

query_database("FROM staging.students;").head()

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S003,78,B
3,S004,88,B
4,S005,95,C


## 13 Subquery

- a query inside of another query (nested?)
- gets a temporary result set, which can be used


In [6]:
query_database("""--sql
SELECT 
    AVG(grade_math)
FROM 
    staging.students
""")

,avg(grade_math)
0,84.2


find all rows where score is higher than the avrage


In [7]:
query_database("""--sql
SELECT 
    * 
FROM 
    staging.students
WHERE grade_math > (
    --subquery
    SELECT 
        AVG(grade_math)
    FROM 
        staging.students       
)
""")

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S004,88,B
3,S005,95,C
4,S009,89,E
5,S010,93,E


## 14 Views


- So far we have saved data inside of tables in a database

  - Regular table:

    | ------------ | ------------ |
    | ------------ | ------------ |
    | ------------ | ------------ |
    | ------------ | ------------ |

- Can also save an SQL qurty as a virtual table (view in ex sakila)
  - Dont save data
  - The query(data) is executed when it is accessed
    - Why is this useful?
      - Simplify complex queries
      - Give this view to end user to have a simple interface
      - Restrict access to certain columns


In [8]:
query_database(""" 
--sql
FROM staging.housing_sales;
""").head()

,sale_id,neighborhood_id,address,date_of_sale,property_type,rooms,living_area_sqm,year_built,sale_price_usd
0,S-24001,1,124 Maple Crest Ave,2024-10-09,Apartment,2,58,2018,245575
1,S-24002,1,9 Birch Hollow Ln,2025-02-21,Townhouse,4,112,2006,454575
2,S-24003,2,77 Riverwalk Way,2025-03-18,Detached,5,168,1999,757625
3,S-24004,2,18 Marina Bluff Ct,2025-07-05,Apartment,3,86,2015,412775
4,S-24005,3,401 Quarry Gate Rd,2024-12-12,Semi-Detached,4,124,1987,381425


pick out avrage of property types and put into a view to givr to downstram users

- if storing this as a table and the underlying data chanches e.g. more houses are added. Then the 'avg table' is wrong
- with a view it will the sql query when the downstram user is queryinh the view - which will give fresh results all the time


In [11]:
query_database("""
--sql
CREATE OR REPLACE VIEW staging.avrage_price_type as 
SELECT 
    property_type,
    AVG(sale_price_usd) as avg_price
FROM staging.housing_sales
GROUP BY property_type;
""")

,Count


In [12]:
query_database("FROM staging.avrage_price_type;")

,property_type,avg_price
0,Semi-Detached,391875.000000
1,Bungalow,423225.000000
2,Townhouse,477913.333333
3,Detached,699105.000000
4,Condo,312803.333333
5,Loft,316112.500000
6,Apartment,290360.714286
7,Terraced,350075.000000


In [ ]:
query_database("FROM information_schema.views")

In [14]:
query_database("FROM information_schema.tables")

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,lecture_13_15,staging,categories,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,lecture_13_15,staging,housing_sales,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,lecture_13_15,staging,neighborhoods,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,lecture_13_15,staging,sales,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,lecture_13_15,staging,students,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,lecture_13_15,staging,teachers,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,lecture_13_15,staging,avrage_price_type,VIEW,None,None,None,None,None,NO,NO,None,None


## Common table expression (CTE)

- use WITH statement
- simplify comlex queries and making them readeble / structured
- temporary result set that you can use further - "variable"


In [17]:
query_database("FROM staging.sales;").head()

,product_id,product_name,category_id,quantity,unit_price_sek
0,P-0001,"Ultrabook 13\"" 8GB/256GB",102,2,11990
1,P-0002,"Gaming Laptop 15\"" 16GB/512GB",102,1,15990
2,P-0003,Office Desktop i5,103,3,7990
3,P-0004,"All-in-One PC 24\""",103,1,9990
4,P-0005,"Android Tablet 10\""",104,2,3490


In [26]:
query_database("""
--sql
WITH avg_sales as (
    -- temp result set that we can use from and use it to construct the query 
    SELECT 
        category_id, 
        AVG(quantity * unit_price_sek)::INT as avg_sales_sek
    FROM staging.sales
    GROUP BY category_id
)
SELECT 
    c.category_name,
    avg_sales_sek 
FROM avg_sales a 
JOIN staging.categories c on c.category_id = a.category_id
ORDER BY avg_sales_sek DESC; 
""")

,category_name,avg_sales_sek
0,Laptops,18820
1,Desktops,15485
2,Phones,7189
3,Furniture,3240
4,Tablets,2929
5,Electronics,2790
6,Appliances,2590
7,Fitness,1563
8,Camping,1059
9,Home & Kitchen,1035
